# Ejercicio 02: Cálculo de la Matriz TF-IDF y Búsqueda de Consultas en un Corpus

El objetivo de este ejercicio es calcular la matriz TF-IDF de un corpus de documentos y luego aplicar una serie de consultas para recuperar los documentos más relevantes. Este ejercicio te ayudará a comprender cómo funciona el modelo de espacio vectorial y cómo se utiliza TF-IDF para ponderar términos en documentos y consultas.

Seguirás los siguientes pasos:

Pasos del Ejercicio

1. Preprocesamiento del texto:
    * Lectura del corpus desde el archivo TXT.
    * Tokenización de los documentos.
    * Normalización del texto (conversión a minúsculas, eliminación de signos de puntuación).
    * Eliminación de palabras vacías (stopwords).

Consultas

Las consultas a aplicar son las siguientes:

    "inteligencia artificial en medicina"
    "beneficios de la educación a distancia"
    "realidad aumentada en videojuegos"
    "desarrollo personal y hábitos saludables"
    "futuro del comercio electrónico"
    "tecnologías en cine moderno"
    "competencias de e-sports"
    "diagnóstico con dispositivos portátiles"
    "literatura de ciencia ficción"
    "plataformas de streaming"

In [4]:
import pandas as pd
import re
import math

ModuleNotFoundError: No module named 'pandas'

In [ ]:
corpus_path = '02tfidfmatrix_corpus.txt'

with open(corpus_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

cleaned_lines = [re.sub(r'^Documento \d+: ', '', line).strip() for line in lines]

data = pd.DataFrame(cleaned_lines, columns=["Document"])

NameError: name 're' is not defined

In [3]:
data

,Document
0,La inteligencia artificial continúa avanzando ...
1,El desarrollo de videojuegos ha alcanzado un n...
2,El comercio electrónico ha cambiado la forma e...
3,La educación a distancia ha permitido a millon...
4,"El cine ha evolucionado con los años, incorpor..."
5,La ciencia ficción es uno de los géneros liter...
6,El desarrollo personal se ha convertido en una...
7,Los avances en la medicina están cambiando la ...


In [4]:
data['Document'][0].lower()

'la inteligencia artificial continúa avanzando rápidamente, transformando sectores como la salud y las finanzas. las empresas están adoptando algoritmos de aprendizaje automático para mejorar la eficiencia. sin embargo, el desafío principal sigue siendo garantizar que las decisiones basadas en datos sean justas y no perpetúen sesgos. la ética es fundamental en este contexto.'

In [5]:
print(data['Document'][0].lower().split())

['la', 'inteligencia', 'artificial', 'continúa', 'avanzando', 'rápidamente,', 'transformando', 'sectores', 'como', 'la', 'salud', 'y', 'las', 'finanzas.', 'las', 'empresas', 'están', 'adoptando', 'algoritmos', 'de', 'aprendizaje', 'automático', 'para', 'mejorar', 'la', 'eficiencia.', 'sin', 'embargo,', 'el', 'desafío', 'principal', 'sigue', 'siendo', 'garantizar', 'que', 'las', 'decisiones', 'basadas', 'en', 'datos', 'sean', 'justas', 'y', 'no', 'perpetúen', 'sesgos.', 'la', 'ética', 'es', 'fundamental', 'en', 'este', 'contexto.']


In [6]:
vocab = set(data['Document'][0].lower().split())
print(vocab)
print(len(vocab))

{'embargo,', 'sigue', 'sin', 'este', 'avanzando', 'aprendizaje', 'adoptando', 'perpetúen', 'inteligencia', 'automático', 'y', 'justas', 'mejorar', 'el', 'garantizar', 'de', 'decisiones', 'salud', 'sesgos.', 'eficiencia.', 'empresas', 'es', 'no', 'desafío', 'transformando', 'principal', 'sectores', 'algoritmos', 'que', 'sean', 'las', 'artificial', 'la', 'para', 'fundamental', 'ética', 'están', 'continúa', 'como', 'siendo', 'basadas', 'en', 'datos', 'finanzas.', 'rápidamente,', 'contexto.'}
46


In [7]:
vocab = []
for i in range(0,len(data)):
    vocab.extend(data['Document'][i].lower().replace(',','').replace('.','').split())

vocab = set(vocab)
print(len(vocab))
vocab = list(vocab)
vocab.sort()
print(vocab)

230
['a', 'acceder', 'accesibles', 'acceso', 'además', 'adopción', 'adoptando', 'ahora', 'alcanzado', 'algoritmos', 'aplicaciones', 'aprendizaje', 'aquellos', 'artificial', 'atraen', 'aumentada', 'aumento', 'aunque', 'automático', 'avances', 'avanzadas', 'avanzando', 'años', 'basadas', 'beneficios', 'bienestar', 'buscan', 'cada', 'calidad', 'cambiado', 'cambiando', 'ciencia', 'cine', 'cinematográficas', 'comercio', 'como', 'competencias', 'completo', 'compramos', 'compras', 'con', 'consume', 'consumidores', 'contenido', 'contexto', 'continúa', 'convertido', 'crear', 'creciente', 'crecimiento', 'cuidados', 'cursos', 'cuándo', 'cómo', 'datos', 'de', 'decisiones', 'demanda', 'democratizando', 'desafían', 'desafío', 'desarrollo', 'desde', 'diagnostican', 'diarias', 'digitales', 'dispositivos', 'distancia', 'donde', 'e-sports', 'educación', 'educativa', 'efectos', 'eficaces', 'eficiencia', 'el', 'electrónico', 'embargo', 'empresas', 'en', 'enfermedades', 'enfoque', 'entretiene', 'es', 'esce

In [8]:
# Lista de stop words en español (se pueden añadir más si es necesario)
stop_words = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las',
    'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
    'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque',
    'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también',
    'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos',
    'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros',
    'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes',
    'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra',
    'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada',
    'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas',
    'algo', 'nosotros', 'mi', 'mis', 'tus', 'te', 'ti', 'tu',
    'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío',
    'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas',
    'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra',
    'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros',
    'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está',
    'estamos', 'estáis', 'están', 'esté', 'estés', 'estemos',
    'estéis', 'estén', 'estaré', 'estarás', 'estará', 'estaremos',
    'estaréis', 'estarán', 'estaría', 'estarías', 'estaríamos',
    'estaríais', 'estarían'
}

vocab = []
for i in range(len(data)):
    words = data['Document'][i].lower().replace(',', '').replace('.', '').split()
    vocab.extend([word for word in words if word not in stop_words])

vocab = sorted(set(vocab))

print(len(vocab))
print(vocab)


199
['acceder', 'accesibles', 'acceso', 'además', 'adopción', 'adoptando', 'ahora', 'alcanzado', 'algoritmos', 'aplicaciones', 'aprendizaje', 'aquellos', 'artificial', 'atraen', 'aumentada', 'aumento', 'aunque', 'automático', 'avances', 'avanzadas', 'avanzando', 'años', 'basadas', 'beneficios', 'bienestar', 'buscan', 'cada', 'calidad', 'cambiado', 'cambiando', 'ciencia', 'cine', 'cinematográficas', 'comercio', 'competencias', 'completo', 'compramos', 'compras', 'consume', 'consumidores', 'contenido', 'contexto', 'continúa', 'convertido', 'crear', 'creciente', 'crecimiento', 'cuidados', 'cursos', 'cuándo', 'cómo', 'datos', 'decisiones', 'demanda', 'democratizando', 'desafían', 'desafío', 'desarrollo', 'diagnostican', 'diarias', 'digitales', 'dispositivos', 'distancia', 'e-sports', 'educación', 'educativa', 'efectos', 'eficaces', 'eficiencia', 'electrónico', 'embargo', 'empresas', 'enfermedades', 'enfoque', 'entretiene', 'es', 'escenas', 'especialmente', 'espectadores', 'estudiantes', 'e

2. Construcción de la matriz TF-IDF:
    * Cálculo de la frecuencia de término (TF) para cada término en cada documento.
    * Cálculo de la frecuencia inversa de documento (IDF) para cada término en el corpus.
    * Cálculo del peso TF-IDF para cada término en cada documento.


In [9]:
def tdtransform(text):
    doc = []
    for word in vocab:
        if word in text.lower().replace(',','').replace('.','').split():
           doc.append(1)
        else:
            doc.append(0)
    return doc

In [10]:
tdmatrix = []
for i in range(0,len(data)):
    doc = tdtransform(text=data['Document'][i])
    tdmatrix.append(doc)

In [11]:
print(tdmatrix)

[[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 

In [12]:
def tftransform(text):
    word_count = {}
    for word in text.lower().replace(',','').replace('.','').split():
        if word in vocab:
            word_count[word] = word_count.get(word, 0) + 1
    
    tf_vector = [word_count.get(word, 0) for word in vocab]
    
    return tf_vector

In [13]:
tfmatrix = []
for i in range(0,len(data)):
    doc = tftransform(text=data['Document'][i])
    tfmatrix.append(doc)

In [14]:
print(tfmatrix)

[[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 

In [15]:
N = len(data)

df = {}

for term in vocab:
    df[term] = sum(1 for doc in data['Document'] if term in doc.lower().replace(',', '').replace('.', '').split())

idf = {term: math.log(N / (1 + df[term])) for term in vocab}

print("IDF values:", idf)

IDF values: {'acceder': 1.3862943611198906, 'accesibles': 1.3862943611198906, 'acceso': 1.3862943611198906, 'además': 1.3862943611198906, 'adopción': 1.3862943611198906, 'adoptando': 1.3862943611198906, 'ahora': 1.3862943611198906, 'alcanzado': 1.3862943611198906, 'algoritmos': 1.3862943611198906, 'aplicaciones': 1.3862943611198906, 'aprendizaje': 0.9808292530117262, 'aquellos': 1.3862943611198906, 'artificial': 0.6931471805599453, 'atraen': 1.3862943611198906, 'aumentada': 1.3862943611198906, 'aumento': 1.3862943611198906, 'aunque': 1.3862943611198906, 'automático': 1.3862943611198906, 'avances': 0.6931471805599453, 'avanzadas': 1.3862943611198906, 'avanzando': 1.3862943611198906, 'años': 1.3862943611198906, 'basadas': 1.3862943611198906, 'beneficios': 1.3862943611198906, 'bienestar': 1.3862943611198906, 'buscan': 1.3862943611198906, 'cada': 1.3862943611198906, 'calidad': 0.9808292530117262, 'cambiado': 1.3862943611198906, 'cambiando': 1.3862943611198906, 'ciencia': 1.3862943611198906

In [16]:
N = len(data)
df = []

for term in vocab:
    term_df = sum(1 for doc in data['Document'] if term in doc.lower().replace(',', '').replace('.', '').split())
    df.append(term_df)

idf = [math.log(N / (1 + term_df)) for term_df in df]

print("IDF values:", idf)
print(len(idf))

IDF values: [1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.9808292530117262, 1.3862943611198906, 0.6931471805599453, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.6931471805599453, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.9808292530117262, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.9808292530117262, 1.3862943611198906, 1.3862943611198906, 1.38629

In [17]:
tfidf_matrix = []

for doc_tf in tfmatrix:
    doc_tfidf = [tf * idf[i] for i, tf in enumerate(doc_tf)]
    tfidf_matrix.append(doc_tfidf)

for row in tfidf_matrix:
    print(row)

[0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 1.3862943611198906, 0.0, 0.9808292530117262, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 1.3862943611198906, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 1.3862943611198906, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 1.3862943611198906, 0.9808292530117262, 0.0, 0.0, 0.0, 0.47000362924573563, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 1.3862943611198906, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9808292530117262, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [18]:
df_tfidf = pd.DataFrame(tfidf_matrix)

In [19]:
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
0,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,1.386294,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,1.386294,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,1.386294,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,2.772589,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.386294,1.386294,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.386294,0.980829,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,0.000000,1.386294,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.980829,0.000000,0.000000,0.000000,0.000000
7,0.000000,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


3. Procesamiento de las consultas:
    * Preprocesamiento de las consultas de manera similar a los documentos.
    * Representación de las consultas en el espacio vectorial TF-IDF.


In [20]:
queries = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

queryBinary = [tftransform(query) for query in queries]

for i, binary in enumerate(queryBinary):
    print(f"Query: {queries[i]}")
    print(f"Binary Representation: {binary}\n")


Query: inteligencia artificial en medicina
Binary Representation: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Query: beneficios de la educación a distancia
Binary Representation: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
queryTFIDF = []

for binary in queryBinary:
    tfidf_vector = [tf * idf[i] for i, tf in enumerate(binary)]
    queryTFIDF.append(tfidf_vector)

for i, tfidf_vector in enumerate(queryTFIDF):
    print(f"Consulta: {queries[i]}")
    print(f"TF-IDF Representation: {tfidf_vector}\n")


Consulta: inteligencia artificial en medicina
TF-IDF Representation: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

4. Cálculo de similitudes:
    * Cálculo de la similitud entre cada consulta y los documentos del corpus utilizando la similitud del coseno.
5. Ranking de documentos:
    * Ordenar los documentos de mayor a menor similitud para cada consulta.
    * Mostrar los documentos más relevantes para cada consulta.

In [22]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(a * b for a, b in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(a**2 for a in vector1))
    magnitude2 = math.sqrt(sum(b**2 for b in vector2))

    if magnitude1 == 0 or magnitude2 == 0:
        return 0  

    return dot_product / (magnitude1 * magnitude2)

similitud = []

for query_vec in queryTFIDF:  # Para cada consulta
    query_similarities = []
    for doc_vec in tfidf_matrix:  # Para cada documento
        sim = cosine_similarity(query_vec, doc_vec)  # Similitud coseno entre consulta y documento
        query_similarities.append(sim)
    similitud.append(query_similarities)

ranking_results = []

for i, query_similarities in enumerate(similitud):
    # Ordenamos los documentos en orden descendente
    ranked_docs = sorted(enumerate(query_similarities), key=lambda x: x[1], reverse=True)
    ranking_results.append(ranked_docs)

for i, ranked_docs in enumerate(ranking_results):
    print(f"Ranking de documentos para la consulta '{queries[i]}':")
    for rank, (doc_index, similarity) in enumerate(ranked_docs[:5], start=1):
        print(f" {rank}. Documento {doc_index + 1} - Similitud: {similarity:.4f}")
    print("\n")


Ranking de documentos para la consulta 'inteligencia artificial en medicina':
 1. Documento 8 - Similitud: 0.3325
 2. Documento 6 - Similitud: 0.0910
 3. Documento 2 - Similitud: 0.0850
 4. Documento 1 - Similitud: 0.0763
 5. Documento 3 - Similitud: 0.0000


Ranking de documentos para la consulta 'beneficios de la educación a distancia':
 1. Documento 4 - Similitud: 0.3349
 2. Documento 1 - Similitud: 0.0000
 3. Documento 2 - Similitud: 0.0000
 4. Documento 3 - Similitud: 0.0000
 5. Documento 5 - Similitud: 0.0000


Ranking de documentos para la consulta 'realidad aumentada en videojuegos':
 1. Documento 2 - Similitud: 0.3294
 2. Documento 5 - Similitud: 0.0554
 3. Documento 1 - Similitud: 0.0000
 4. Documento 3 - Similitud: 0.0000
 5. Documento 4 - Similitud: 0.0000


Ranking de documentos para la consulta 'desarrollo personal y hábitos saludables':
 1. Documento 7 - Similitud: 0.3766
 2. Documento 2 - Similitud: 0.0557
 3. Documento 1 - Similitud: 0.0000
 4. Documento 3 - Similitud: